# Test uv-theory WCA implementation

In [1]:
from feos_uvtheory import *
from feos_uvtheory.si import *
from feos_uvtheory.eos import *
import numpy as np

**1. Pure fluids**

In [2]:
diameters = np.array([3.7039]) # in Angstrom
eps_k = np.array([150.03]) # in Kelvin


## WCA ##
parameters_wca = UVParameters.from_lists(np.array([12.0]), np.array([6.0]), diameters, eps_k)
eos_wca = UVTheory(parameters_wca, perturbation=Perturbation.WeeksChandlerAndersen, max_eta=0.5)

## BH ##
parameters_bh = UVParameters.from_lists(np.array([24.0]), np.array([6.0]), diameters, eps_k)
eos_bh = UVTheory(parameters_bh, perturbation=Perturbation.BarkerHenderson)

## State ##
dimensionless_temperature = 4.0
dimensionless_density = 1.0
temperature = dimensionless_temperature * eps_k[0] * KELVIN
density = dimensionless_density / (diameters[0]**3 * ANGSTROM**3 * NAV)


state_wca = State(eos_wca, temperature=temperature, density=density, molefracs = np.array([1.0]))
state_bh = State(eos_bh, temperature=temperature, density=density, molefracs = np.array([1.0]))

In [3]:
a_wca = state_wca.helmholtz_energy(Contributions.Residual) / (KB * temperature)
a_bh = state_bh.helmholtz_energy(Contributions.Residual) / (KB * temperature)
a_thijs_wca = 1.973655316
a_thijs_bh = 2.993577297
print('      |Result Rust Implementation| Result Thijs | Difference')
print('  WCA |{}        | {}  |{}'.format(a_wca, a_thijs_wca, np.abs(a_wca - a_thijs_wca)))
print('  BH  |{}        | {}  |{}'.format(a_bh, a_thijs_bh, np.abs(a_bh - a_thijs_bh)))

      |Result Rust Implementation| Result Thijs | Difference
  WCA |1.9736553254042635        | 1.973655316  |9.404263368040233e-09
  BH  |2.9935773057794126        | 2.993577297  |8.779412752346616e-09


**2. Mixtures**

Test 1: "Mixture" of equal components

In [4]:
diameters = np.array([3.7039, 3.7039]) # in Angstrom
eps_k = np.array([150.03, 150.03]) # in Kelvin

## WCA ##
parameters_wca = UVParameters.from_lists(np.array([12.0, 12.0]), np.array([6.0, 6.0]), diameters, eps_k)
eos_wca = UVTheory(parameters_wca, perturbation=Perturbation.WeeksChandlerAndersen)

## BH ##
parameters_bh = UVParameters.from_lists(np.array([24.0, 24.0]), np.array([6.0, 6.0]), diameters, eps_k)
eos_bh = UVTheory(parameters_bh, perturbation=Perturbation.BarkerHenderson)

## State ##
dimensionless_temperature = 4.0
dimensionless_density = 1.0
temperature = dimensionless_temperature * eps_k[0] * KELVIN
density = dimensionless_density / (diameters[0]**3 * ANGSTROM**3 * NAV)


state_wca = State(eos_wca, temperature=temperature, density=density, molefracs = np.array([0.7, 0.3]))
a_wca = state_wca.helmholtz_energy(Contributions.Residual) / (KB * temperature)

state_bh = State(eos_bh, temperature=temperature, density=density, molefracs = np.array([0.7, 0.3]))
a_bh = state_bh.helmholtz_energy(Contributions.Residual) / (KB * temperature)

a_thijs_wca = 1.973655316
a_thijs_bh = 2.993577297
print('      |Result Rust Implementation| Result Thijs | Difference')
print('  WCA |{}        | {}  |{}'.format(a_wca, a_thijs_wca, np.abs(a_wca - a_thijs_wca)))
print('  BH  |{}        | {}  |{}'.format(a_bh, a_thijs_bh, np.abs(a_bh - a_thijs_bh)))

      |Result Rust Implementation| Result Thijs | Difference
  WCA |1.973655325404259        | 1.973655316  |9.404258927148135e-09
  BH  |2.993577305779411        | 2.993577297  |8.779410975989776e-09


Test 2: Different energy parameters

In [23]:
diameters = np.array([1.0, 1.0]) # in Angstrom
eps_k = np.array([1.0, 0.5]) # in Kelvin
## State ##
dimensionless_temperature = 1.0
dimensionless_density = 0.9
temperature = dimensionless_temperature * eps_k[0] * KELVIN
density = dimensionless_density / (diameters[0]**3 * ANGSTROM**3 * NAV)

## WCA Results Thijs for comparison ###
a0 = 4.1719593852430794 
ahs = 3.8636904888563084
delta_a0 = a0 - ahs
Delta_B2 = -4.7846399638747954
a1u = -4.7678301069070645
phi_u =  0.99750066585468078
a_disp = -4.7697504236074844
a = a0 + a_disp

## WCA ##
parameters_wca = UVParameters.from_lists(np.array([12.0, 12.0]), np.array([6.0, 6.0]), diameters, eps_k)
eos_wca = UVTheory(parameters_wca, perturbation=Perturbation.WeeksChandlerAndersen)


state_wca = State(eos_wca, temperature=temperature, density=density, molefracs = np.array([0.4, 0.6]))
a_wca = state_wca.helmholtz_energy(Contributions.Residual) / (KB * temperature)


a_thijs_wca = -4.7697504236074844 + 4.1719593852430794
print('          |Result Rust Implementation| Result Thijs | Difference')
print('  HS      |{}        | {}  |{}'.format(state_wca.helmholtz_energy_contributions()[1][1] / (KB * temperature) , ahs, np.abs(state_wca.helmholtz_energy_contributions()[1][1] / (KB * temperature) - ahs)))
print(' Delta a0 |{}        | {}  |{}'.format(state_wca.helmholtz_energy_contributions()[2][1] / (KB * temperature), delta_a0, np.abs(state_wca.helmholtz_energy_contributions()[2][1] / (KB * temperature) - delta_a0)))
print(' a_disp   |{}        | {}  |{}'.format(state_wca.helmholtz_energy_contributions()[3][1] / (KB * temperature), a_disp, np.abs(state_wca.helmholtz_energy_contributions()[3][1] / (KB * temperature) - a_disp)))
print(' Full WCA |{}        | {}  |{}'.format(a_wca, a, np.abs(a_wca - a)))

          |Result Rust Implementation| Result Thijs | Difference
  HS      |3.863690488856311        | 3.8636904888563084  |2.6645352591003757e-15
 Delta a0 |0.30826889687807507        | 0.308268896386771  |4.913040529963553e-10
 a_disp   |-4.769750221478107        | -4.769750423607484  |2.02129377235849e-07
 Full WCA |-0.597790835743721        | -0.597791038364405  |2.0262068389786947e-07


Test 3: Different energy parameters and diameters

In [24]:
diameters = np.array([1.0, 2.0]) # in Angstrom
eps_k = np.array([1.0, 0.5]) # in Kelvin

## WCA ##
parameters_wca = UVParameters.from_lists(np.array([12.0, 12.0]), np.array([6.0, 6.0]), diameters, eps_k)
eos_wca = UVTheory(parameters_wca, perturbation=Perturbation.WeeksChandlerAndersen)

## State ##
dimensionless_temperature = 1.5
dimensionless_density = 0.52
temperature = dimensionless_temperature * eps_k[0] * KELVIN
sigma_x_3 = 0.4 * diameters[0]**3 + 0.6 * diameters[1]**3
density = dimensionless_density / (sigma_x_3 * ANGSTROM**3 * NAV)


state_wca = State(eos_wca, temperature=temperature, density=density, molefracs = np.array([0.4, 0.6]))
a_wca = state_wca.helmholtz_energy(Contributions.Residual) / (KB * temperature)

a = 1.2976589884809138 - 1.3318659166866607

a_thijs_wca = -4.7697504236074844 + 4.1719593852430794
print('      |Result Rust Implementation| Result Thijs | Difference')
print('  WCA |{}        | {}  |{}'.format(a_wca, a, np.abs(a_wca - a)))

      |Result Rust Implementation| Result Thijs | Difference
  WCA |-0.03420620736313928        | -0.03420692820574689  |7.208426076113494e-07
